In [1]:
import os
import json
import pandas as pd
from glob import glob
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import time
import random
from itertools import chain
from tqdm import tqdm

# Data Annotation Check

In [2]:
#경로 지정
origin_path='C:/Users/kwonc/Desktop/데이터제출용'

In [ ]:
class DAnn_check():
    def __init__(self,img_root,label_root):
        self.img_root=img_root #C:/Users/kwonc/Desktop/test/원천데이터
        self.label_root=label_root #C:/Users/kwonc/Desktop/test/라벨링데이터
        
    def check(self):
        random_seed=42
        random.seed(random_seed)
        min_dict_B={}
        print('백묵명 최소 Annotation 개수 추출')
        
        for i in os.listdir(self.img_root+'/'+'실제데이터'+'/'+'Bounding Box'):
            # logger.info('백묵병 데이터의 최소 Annotation 개수를 측정합니다')
            if (i=='02.생애이슈(백묵병_음성)')|(i=='12.생애이슈(백묵병)'):
                print(i)
                min_dict_B[f'num_of_{i}']=0
                min_dict_B[f'num_of_{i}_file']=0
               
                for j in tqdm(os.listdir(self.img_root+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+i)):
                    with open(self.label_root+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+i+'/'+j.split('.')[0]+'.json','r',encoding='UTF-8') as Bounding_d:
                        json_file_D=json.load(Bounding_d)
                        min_dict_B[f'num_of_{i}'] += len(json_file_D['ANNOTATION_INFO'])
                        min_dict_B[f'num_of_{i}_file']+=1
                        #print(min_dict_B)
                        num_of_min_ann_Bounding=min(min_dict_B.values())
            else:
                pass
        print(f'백묵병, 음성, 양성 데이터의 최소 Annotation 개수는 {num_of_min_ann_Bounding}개 입니다.')
        # logger.info(f'백묵병 음성, 양성 데이터의 최소 Annotation 개수는 {num_of_min_ann_Bounding}개 입니다.')
        return num_of_min_ann_Bounding

In [ ]:
num_of_min_ann_D_Bounding=DAnn_check(img_root=origin_path,label_root=label_path).check()

# Data Loading

In [ ]:
#백묵병 음성, 양성 데이터 Load
class D_Load():
    def __init__(self,img_root,label_root,class_list,Ann_min,resize=112):
        self.img_root=img_root
        self.label_root=label_root
        self.class_list=class_list
        self.Ann_min=Ann_min
        self.trans_tensor = transforms.ToTensor()
        self.trans_resize = transforms.Resize((resize, resize))
        
    def D_Loaddata(self):
        random_seed=42
        random.seed(random_seed)
        
        data_dict_D={}
        label_dict_D={} #LifeCycle_disease
        return_list_D=[]
        
        #2.애벌레, 3.번데기, 12.애벌레-백묵병, 13.번데기-백묵병
        for i in os.listdir(self.img_root+'/'+'실제데이터'+'/'+'Bounding Box'): #i는 클래스명
            if (i=='02.생애이슈(백묵병_음성)')|(i=='12.생애이슈(백묵병)'):
                print(f'{i}_Loading 시작_백묵병 classification')
                # logger.info(f'{i}_Loading 시작.')
                data_dict_D[f'{i}']=list(range(self.Ann_min))
                class_file_list=os.listdir(self.img_root+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+i)
                
                
                if self.Ann_min <= len(class_file_list):
                    n=0
                    rs=random.sample(class_file_list,self.Ann_min)
                    
                    for k in tqdm(rs):
                        #print(k)
                        try:
                            with open(self.label_root+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+i+'/'+k.split('.')[0]+'.json','r',encoding='UTF-8') as f:
                                jsonfile_D=json.load(f)
                                jsonfile_D['ANNOTATION_INFO']=jsonfile_D['ANNOTATION_INFO'].pop(random.randrange(0,len(jsonfile_D['ANNOTATION_INFO'])))
                                
                                #음성, 양성 0,1->NEG,POS로 바꾸기
                                if jsonfile_D['ANNOTATION_INFO']['DISEASES']==0:
                                    jsonfile_D['ANNOTATION_INFO']['DISEASES']='NEG' #음성
                                elif jsonfile_D['ANNOTATION_INFO']['DISEASES']==1:
                                    jsonfile_D['ANNOTATION_INFO']['DISEASES']='POS' #양성
                                else: #나머지는 pass
                                    pass
                                #print(jsonfile_D['ANNOTATION_INFO']['DISEASES'])
                                
                                label_dict_D[jsonfile_D['IMAGE']['IMAGE_FILE_NAME']]=self.class_list.index(jsonfile_D['ANNOTATION_INFO']['LIFECYCLE']+'_'+jsonfile_D['ANNOTATION_INFO']['DISEASES'])
                                #print(label_dict_D)
                                
                                x_data=self.trans_tensor(
                                    self.trans_resize(
                                        Image.open(self.img_root+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+i+'/'+k).crop(
                                            (jsonfile_D['ANNOTATION_INFO']['XTL'],jsonfile_D['ANNOTATION_INFO']['YTL'],jsonfile_D['ANNOTATION_INFO']['XBR'],jsonfile_D['ANNOTATION_INFO']['YBR']))))
                                
                                y_data=label_dict_D[jsonfile_D['IMAGE']['IMAGE_FILE_NAME']]
                                data_dict_D[f'{i}'][n] =[x_data, y_data, jsonfile_D['IMAGE']['IMAGE_FILE_NAME']]
                                n += 1
                        except:
                            pass
                        
                else: #여기로 넘어가지 않음
                    n=0
                    rest_json_D={}
                    for o in tqdm(class_file_list):
                        #print(o)
                        try:
                            with open(self.label_root+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+i+'/'+o.split('.')[0]+'.json','r',encoding='UTF-8') as t:
                                jsonfile_D=json.load(t)
                                label_dict_D[jsonfile_D['IMAGE']['IMAGE_FILE_NAME']]=self.class_list.index(jsonfile_D['ANNOTATION_INFO'][0]['LIFECYCLE']+'_'+jsonfile_D['ANNOTATION_INFO']['DISEASES'])
                                
                                #모든 이미지에 대해서 1개의 Annotation 진행
                                pop_num=random.choice(jsonfile_D['ANNOTATION_INFO'])
                                crop_ann=jsonfile_D['ANNOTATION_INFO'].pop(jsonfile_D['ANNOTATION_INFO'].index(pop_num))
                                rest_json_D[jsonfile_D['IMAGE']['IMAGE_FILE_NAME']] = jsonfile_D['ANNOTATION_INFO']
                                
                                if len(rest_json_D[jsonfile_D['IMAGE']['IMAGE_FILE_NAME']])==0:
                                    del rest_json_D[jsonfile_D['IMAGE']['IMAGE_FILE_NAME']]
                                    
                                x_data = self.trans_tensor(
                                    self.trans_resize(
                                        Image.open(self.img_root+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+i+'/'+o).crop(
                                            (crop_ann['XTL'],crop_ann['YTL'],crop_ann['XBR'],crop['YBR'])
                                        )
                                    )
                                )
                                
                                y_data=label_dict_D[jsonfile_D['IMAGE']['IMAGE_FILE_NAME']] #lifecycle+disease
                                data_dict_D[f'{i}'][n]=[x_data,y_data,jsonfile_D['IMAGE']['IMAGE_FILE_NAME']]
                                n += 1
                                
                        except:
                            pass
                        
                        
                        while(len(list(chain(*rest_json_D.values()))) != (self.Ann_min-len(class_file_list))):
                            del_key=random.choice(list(rest_json_D.keys()))
                            
                            if len(rest_json_D[del_key]) > 0:
                                rest_json_D[del_key].pop(rest_json_D[del_key].index(random.choice(rest_json_D[del_key])))
                                if len(rest_json_D[del_key])==0:
                                    del rest_json_D[del_key]
                                    
                        for p in rest_json_D.keys():
                            for ann in range(len(rest_json_D[p])):
                                x_data=self.trans_tensor(
                                    self.trans_resize(
                                        Image.open(self.img_root+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+i+'/'+p).crop(
                                            (rest_json_D[p][ann]['XTL'],rest_json_D[p][ann]['YTL'],rest_json_D[p][ann]['XBR'],rest_json_D[p][ann]['YBR'])
                                        )
                                    )
                                )
                                
                                y_data=label_dict_D[p]
                                data_dict_D[f'{i}'][n]=[x_data,y_data,p]
                                n+=1
                                
                print(f'{i}_Loading 완료. {i}은(는) {n}개의 Annotation이 로드 되었습니다.')
                # logger.info(f'{i}_Loading 완료. {i}은(는) {n}개의 Annotation이 로드 되었습니다.')
            else:
                pass
            
        for u in data_dict_D.keys():
            return_list_D += data_dict_D[u]
            
        return return_list_D    

In [ ]:
dataset=D_Load(img_root=origin_path,label_root=label_path,class_list=['LA_NEG','LA_POS'],Ann_min=num_of_min_ann_D_Bounding).D_Loaddata()

# Data Split

In [ ]:
class_number=2
random_seed=42
torch.manual_seed(random_seed)
ratio=[0.8, 0.1, 0.1]

traindataset = []
valdataset = []
testdataset = []

for data_type in ['dataset']:
    data_len = len(globals()[f'{data_type}'])
    dataset = globals()[f'{data_type}']
    class_list_n = [i for i in range(class_number)]
    idx_count = {i : 0 for i in range(class_number)}
    
    for i in range(len(class_list_n)):
        globals()[f'c_dataset_{i}'] = []
        globals()[f'dataset_{i}'] = []
        globals()[f'traindataset_{i}'] = []
        globals()[f'valdataset_{i}'] = []
        globals()[f'testdataset_{i}'] = []

    for idx in range(data_len):
        idx_count[dataset[idx][1]] += 1

        globals()[f'c_dataset_{dataset[idx][1]}'] += [dataset[idx]]

    train_size = int(np.ceil(min(idx_count.values()) * ratio[0]))
    val_size = int(min(idx_count.values()) * ratio[1])
    test_size = int(min(idx_count.values())) - train_size - val_size

    for j in range(len(class_list_n)):
        globals()[f'dataset_{j}'] = random.sample(globals()[f'c_dataset_{j}'], min(idx_count.values()))
        globals()[f'traindataset_{i}'], globals()[f'valdataset_{i}'], globals()[f'testdataset_{i}'] = torch.utils.data.random_split(globals()[f'dataset_{j}'], [train_size, val_size, test_size])
        traindataset += globals()[f'traindataset_{i}']
        valdataset += globals()[f'valdataset_{i}']
        testdataset += globals()[f'testdataset_{i}']

print(f'분할된 Traindataset은 {len(traindataset)}개, Valdataset은 {len(valdataset)}개, Testdataset은 {len(testdataset)}개 입니다.', '(', datetime.datetime.now(), ')')
# logger.info(f'분할된 Traindataset은 {len(traindataset)}개, Valdataset은 {len(valdataset)}개, Testdataset은 {len(testdataset)}개 입니다.')
print(f'설정한 Train : Validation : Test 비율은 {ratio[0]} : {ratio[1]} : {ratio[2]} 입니다.', '(', datetime.datetime.now(), ')')
# logger.info(f'설정한 Train : Validation : Test 비율은 {ratio[0]} : {ratio[1]} : {ratio[2]} 입니다.')
print('Train, Validation, Test 분할이 완료 되었습니다.', '(', datetime.datetime.now(), ')')
# logger.info('Train, Validation, Test 분할이 완료 되었습니다.')

# Data folder Move

In [ ]:
# Train

image_data_file_name=[]
label_data_file_name=[]


for i in range(len(traindataset)):
    ID=traindataset[i][2].split('.')[0]+'.json'
    label_data_file_name.append(ID)
    
for i in range(len(traindataset)):
    ID=traindataset[i][2]
    image_data_file_name.append(ID)
    
#logger.info('Test에 사용한 이미지, Json파일을 검증용 데이터 셋 파일에 복사합니다.')

for i in tqdm(range(len(traindataset))):
    if 'AA' in image_data_file_name[i]:
        shutil.copy(origin_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'12.생애이슈(백묵병)'+'/'+image_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/train/원천데이터/실제데이터/Bounding Box/12.생애이슈(백묵병_양성)')
        shutil.copy(label_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'12.생애이슈(백묵병)'+'/'+label_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/train/라벨링데이터/실제데이터/Bounding Box/12.생애이슈(백묵병_양성)')
    elif 'LA' in image_data_file_name[i]:
        shutil.copy(origin_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'02.생애이슈(백묵병_음성)'+'/'+image_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/train/원천데이터/실제데이터/Bounding Box/02.생애이슈(백묵병_음성)')
        shutil.copy(label_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'02.생애이슈(백묵병_음성)'+'/'+label_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/train/라벨링데이터/실제데이터/Bounding Box/02.생애이슈(백묵병_음성)')
    else:
        pass
    
#logger.info('Test에 사용한 데이터를 검증용 데이터 셋 파일에 저장 완료하였습니다.')

In [ ]:
# Valid

image_data_file_name=[]
label_data_file_name=[]


for i in range(len(valdataset)):
    ID=valdataset[i][2].split('.')[0]+'.json'
    label_data_file_name.append(ID)
    
for i in range(len(valdataset)):
    ID=valdataset[i][2]
    image_data_file_name.append(ID)
    
#logger.info('Test에 사용한 이미지, Json파일을 검증용 데이터 셋 파일에 복사합니다.')

for i in tqdm(range(len(valdataset))):
    if 'AA' in image_data_file_name[i]:
        shutil.copy(origin_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'12.생애이슈(백묵병)'+'/'+image_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/valid/원천데이터/실제데이터/Bounding Box/12.생애이슈(백묵병_양성)')
        shutil.copy(label_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'12.생애이슈(백묵병)'+'/'+label_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/valid/라벨링데이터/실제데이터/Bounding Box/12.생애이슈(백묵병_양성)')
    elif 'LA' in image_data_file_name[i]:
        shutil.copy(origin_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'02.생애이슈(백묵병_음성)'+'/'+image_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/valid/원천데이터/실제데이터/Bounding Box/02.생애이슈(백묵병_음성)')
        shutil.copy(label_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'02.생애이슈(백묵병_음성)'+'/'+label_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/valid/라벨링데이터/실제데이터/Bounding Box/02.생애이슈(백묵병_음성)')
    else:
        pass
    
#logger.info('Test에 사용한 데이터를 검증용 데이터 셋 파일에 저장 완료하였습니다.')

In [ ]:
# Test

image_data_file_name=[]
label_data_file_name=[]


for i in range(len(testdataset)):
    ID=testdataset[i][2].split('.')[0]+'.json'
    label_data_file_name.append(ID)
    
for i in range(len(testdataset)):
    ID=testdataset[i][2]
    image_data_file_name.append(ID)
    
#logger.info('Test에 사용한 이미지, Json파일을 검증용 데이터 셋 파일에 복사합니다.')

for i in tqdm(range(len(testdataset))):
    if 'AA' in image_data_file_name[i]:
        shutil.copy(origin_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'12.생애이슈(백묵병)'+'/'+image_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/test/원천데이터/실제데이터/Bounding Box/12.생애이슈(백묵병_양성)')
        shutil.copy(label_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'12.생애이슈(백묵병)'+'/'+label_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/test/라벨링데이터/실제데이터/Bounding Box/12.생애이슈(백묵병_양성)')
    elif 'LA' in image_data_file_name[i]:
        shutil.copy(origin_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'02.생애이슈(백묵병_음성)'+'/'+image_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/test/원천데이터/실제데이터/Bounding Box/02.생애이슈(백묵병_음성)')
        shutil.copy(label_path+'/'+'실제데이터'+'/'+'Bounding Box'+'/'+'02.생애이슈(백묵병_음성)'+'/'+label_data_file_name[i],'C:/Users/kwonc/Desktop/데이터제출용/test/라벨링데이터/실제데이터/Bounding Box/02.생애이슈(백묵병_음성)')
    else:
        pass
    
#logger.info('Test에 사용한 데이터를 검증용 데이터 셋 파일에 저장 완료하였습니다.')

In [3]:
# 이미지 데이터 Resize->현재 데이터는 1920X1080 Size임
# Resize 해줄때만 사용함.

# for i in os.listdir('c:/Users/kwonc/Desktop/Docker/dataset/export/images'):
#     img=Image.open('c:/Users/kwonc/Desktop/Docker/dataset/export/images'+'/'+i)
#     img_resize=img.resize((640,480))
#     img_resize.save('c:/Users/kwonc/Desktop/Docker/dataset/export/images'+'/'+i)

In [3]:
### Yolov5 형식에 맞게 폴더 구조 및 라벨링 데이터 변환

In [8]:
# Train dataset Count
print('Train의 백묵병 양성 이미지 데이터 개수는:',len(os.listdir('C:/Users/kwonc/Desktop/데이터제출용/train/원천데이터/실제데이터/Bounding Box/12.생애이슈(백묵병_양성)')))
print('Train의 백묵병 양성 라벨링 데이터 개수는:',len(os.listdir('C:/Users/kwonc/Desktop/데이터제출용/train/라벨링데이터/실제데이터/Bounding Box/12.생애이슈(백묵병_양성)')))
print('Train의 백묵병 음성 이미지 개수는:',len(os.listdir('C:/Users/kwonc/Desktop/데이터제출용/train/원천데이터/실제데이터/Bounding Box/02.생애이슈(백묵병_음성)')))
print('Train의 백묵병 음성 이미지 개수는:',len(os.listdir('C:/Users/kwonc/Desktop/데이터제출용/train/라벨링데이터/실제데이터/Bounding Box/02.생애이슈(백묵병_음성)')))


Train의 백묵병 양성 이미지 데이터 개수는: 8400
Train의 백묵병 양성 라벨링 데이터 개수는: 8400
Train의 백묵병 음성 이미지 개수는: 8400
Train의 백묵병 음성 이미지 개수는: 8400


In [4]:
# img=Image.open('c:/Users/kwonc/Desktop/Docker/dataset/export/images'+'/'+'01_1_R_LA_NA_20220707_01_003.jpg')
# img.size

(640, 480)

In [8]:

for i in tqdm(os.listdir('c:/Users/kwonc/Desktop/Docker/dataset/export/labels')):
    label_disease={}
    img=i.split('.')[0]
    with open('c:/Users/kwonc/Desktop/Docker/dataset/export/labels/'+i,'r',encoding='UTF-8') as f:
        jsonfile=json.load(f)
    for j in range(len(jsonfile['ANNOTATION_INFO'])):
        value=str(jsonfile['ANNOTATION_INFO'][j]['DISEASES'])
        X_center=((jsonfile['ANNOTATION_INFO'][j]['XTL']/1920)+(jsonfile['ANNOTATION_INFO'][j]['XBR']/1920))/2 #왼쪽위 X좌표
        Y_center=((jsonfile['ANNOTATION_INFO'][j]['YTL']/1080)+(jsonfile['ANNOTATION_INFO'][j]['YBR']/1080))/2 #왼쪽위 Y좌표
        Width=((jsonfile['ANNOTATION_INFO'][j]['XBR'])-(jsonfile['ANNOTATION_INFO'][j]['XTL']))/1920 #오른쪽 아래 x좌표
        Height=((jsonfile['ANNOTATION_INFO'][j]['YBR'])-(jsonfile['ANNOTATION_INFO'][j]['YTL']))/1080 #오른쪽 아래 y좌표
        label_disease[f'{j}']=[value,X_center,Y_center,Width,Height]
        data=pd.DataFrame(label_disease).T
        data.to_csv(f'c:/Users/kwonc/Desktop/Docker/dataset/export/labels/{img}.txt',sep=' ',index=False,header=False)

print('라벨값 정리 끝')

100%|██████████| 21000/21000 [13:33<00:00, 25.82it/s]

라벨값 정리 끝


In [9]:
# %cd c:/Users/kwonc/Desktop/Docker/yolov5
# %pip install -r requirements.txt

In [15]:
img_list=os.listdir('c:/Users/kwonc/Desktop/Docker/dataset/export/images')

In [16]:
img_list_file=[]
for i in img_list:
    img_file='c:/Users/kwonc/Desktop/Docker/dataset/export/images'+'/'+i
    img_list_file.append(img_file)

In [17]:
train_img, val_img= train_test_split(img_list_file,test_size=0.2,random_state=42)

In [18]:
print(len(train_img))

80


In [20]:
val_img

['c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_168.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_098.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_128.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_089.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_084.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_076.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_046.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_162.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_024.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_003.jpg',
 'c:/Users/kwonc/Desktop/Docker/dataset/export/images/01_1_R_LA_NA_20220707_01_041.jpg',
 'c:/Users/kwonc/Desk

In [22]:
#txt파일 생성

with open('c:/Users/kwonc/Desktop/Docker/dataset/train.txt','w') as f:
    f.write('\n'.join(train_img)+'\n')

with open('c:/Users/kwonc/Desktop/Docker/dataset/val.txt','w') as f:
    f.write('\n'.join(val_img)+'\n')


In [23]:
import yaml

with open('c:/Users/kwonc/Desktop/Docker/dataset/data.yaml','r') as f:
    data=yaml.load(f,Loader=yaml.FullLoader)

print(data)


{'names': ['No disease', 'disease'], 'nc': 2, 'train': 'c:/Users/kwonc/Desktop/Docker/dataset/train.txt', 'val': 'c:/Users/kwonc/Desktop/Docker/dataset/val.txt'}


In [24]:
data['train']='c:/Users/kwonc/Desktop/Docker/dataset/train.txt'
data['val']='c:/Users/kwonc/Desktop/Docker/dataset/val.txt'

with open('c:/Users/kwonc/Desktop/Docker/dataset/data.yaml','w') as f:
    yaml.dump(data,f)

print(data)

{'names': ['No disease', 'disease'], 'nc': 2, 'train': 'c:/Users/kwonc/Desktop/Docker/dataset/train.txt', 'val': 'c:/Users/kwonc/Desktop/Docker/dataset/val.txt'}


In [25]:
%cd c:/Users/kwonc/Desktop/Docker/yolov5/

c:\Users\kwonc\Desktop\Docker\yolov5


In [28]:
!python train.py --img 640 --batch 2 --epochs 3 --data c:/Users/kwonc/Desktop/Docker/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name bee_yolov5s_result

wandb: Currently logged in as: eumty13. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=c:/Users/kwonc/Desktop/Docker/dataset/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=3, batch_size=2, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=bee_yolov5s_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github:  YOLOv5 is out of date by 7 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5  v7.0-33-g1752768 Python-3.10.7 torch-1.13.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005,


wandb:  View run at https://wandb.ai/eumty13/train/runs/1h1bf8rl
Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  mod